<h3>Vytvoření y</h3>

y = co má neuronka předpovídat. V našem případě začneme tím, zda byl v daném čase L-mod nebo H-mod.

Je třeba:

- načíst signály popisující začátek a konec H-módu v daném výboji: `t_H_mode_start` a `t_H_mode_end`.

- z adresáře s obrazovými daty zjistit časy snímků kamery v daném výboji

- pro každý čas zjistit, zda jde o L-mód nebo H-mód

- uložit do csv (např ve formátu slupců [název souboru se snímkem z kamery, čas ve výboji, label zda L-mode/H-mode]) - použijte pandas

Importing libraries

In [1]:
import os
os.chdir("/compass/Shared/Users/bogdanov/vyzkumny_ukol")
from cdb_extras import xarray_support as cdbxr   # načítání dat z databáze COMPASSu
from pyCDB import client
import numpy as np         # práce s numerickými poli
import pandas as pd
from pathlib import Path
%matplotlib inline
from tqdm.auto import tqdm

#import module with functions from the tutorial
import imgs_processing as img
cdb = client.CDBClient()

### Importing data

In [2]:
shot = 16769 # 18057]
out_path = Path('./imgs')
ris1_data = img.load_RIS_data(shot, 1)
ris2_data = img.load_RIS_data(shot, 2)

### Saving the images with plasma on it

In [3]:
ris1_names = img.save_ris_images_to_folder(ris1_data, path=out_path, ris=1, shot=shot, use_discharge_duration=True, just_names=False)
ris2_names = img.save_ris_images_to_folder(ris2_data, path=out_path, ris=2, shot=shot, use_discharge_duration=True, just_names=False)

Demosaicing images


Saving images:   0%|          | 0/2178 [00:00<?, ?it/s]

Demosaicing images


Saving images:   0%|          | 0/2178 [00:00<?, ?it/s]

### Create a `pd.DataFrame` containing time, mode, and image path

In [4]:
LorH = pd.DataFrame(data={'mode':np.full(len(ris1_data), 'L-mode')}, index=pd.Index(ris1_data.time, name='time'))
#contains time and the state of the plasma (L-mode, H-mode, ELM)

t_ELM_start = cdb.get_signal(f"t_ELM_start/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_ELM_end = cdb.get_signal(f"t_ELM_end/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_H_mode_start = cdb.get_signal(f"t_H_mode_start/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_H_mode_end = cdb.get_signal(f"t_H_mode_end/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")

#TODO:  To create a DataFrame with only one row, you need to specify an index, 
# so if plasma enters H-mode more than once during one shot, then next row creates different shape

try:
    len(t_ELM_start.data)
except:
    t_ELM = pd.DataFrame({'start':t_ELM_start.data, 'end':t_ELM_end.data}, index=[0])
else:
    t_ELM = pd.DataFrame({'start':t_ELM_start.data, 'end':t_ELM_end.data})

try:
    len(t_H_mode_start.data)
except:
    t_H_mode = pd.DataFrame({'start':t_H_mode_start.data, 'end':t_H_mode_end.data}, index=[0])
else:
    t_H_mode = pd.DataFrame({'start':t_H_mode_start.data, 'end':t_H_mode_end.data})


for H_mode in t_H_mode.values:
    LorH.loc[H_mode[0]:H_mode[1]] = 'H-mode'

for elm in t_ELM.values:
    LorH.loc[elm[0]:elm[1]] = 'ELM'

#Discarding pictures without plasma
discharge_start, discharge_end = img.discharge_duration(shot)
LorH = LorH[discharge_start : discharge_end]

#Appending columns with paths of the RIS imgs
LorH['filename'] = np.array(ris1_names)

LorH.to_csv(f'/compass/Shared/Users/bogdanov/vyzkumny_ukol/LHmode-detection-shot{shot}.csv')

## Process multiple shots

In [6]:
out_path = Path('./imgs')
for shot in [19083, 19094, 19241, 20009]:#, 20097, 20112, 20143, 20144, 20145, 20146, 20147]:
    print('Working on shot ', shot)
    out_path = Path('./imgs')
    ris1_data = img.load_RIS_data(shot, 1)
    ris2_data = img.load_RIS_data(shot, 2)
    ris1_names = img.save_ris_images_to_folder(ris1_data, path=out_path, ris=1, shot=shot, use_discharge_duration=True, just_names=False)
    ris2_names = img.save_ris_images_to_folder(ris2_data, path=out_path, ris=2, shot=shot, use_discharge_duration=True, just_names=False)
    LorH = pd.DataFrame(data={'mode':np.full(len(ris1_data), 'L-mode')}, index=pd.Index(ris1_data.time, name='time'))
    #contains time and the state of the plasma (L-mode, H-mode, ELM)

    t_ELM_start = cdb.get_signal(f"t_ELM_start/SYNTHETIC_DIAGNOSTICS:{shot}:seidl_2023")
    t_ELM_end = cdb.get_signal(f"t_ELM_end/SYNTHETIC_DIAGNOSTICS:{shot}:seidl_2023")
    t_H_mode_start = cdb.get_signal(f"t_H_mode_start/SYNTHETIC_DIAGNOSTICS:{shot}:seidl_2023")
    t_H_mode_end = cdb.get_signal(f"t_H_mode_end/SYNTHETIC_DIAGNOSTICS:{shot}:seidl_2023")

    #TODO:  To create a DataFrame with only one row, one needs to specify an index, 
    # so if plasma enters H-mode more than once during one shot index have to be passed. Thus crutch with try: except:
    try:
        len(t_ELM_start.data)
    except:
        t_ELM = pd.DataFrame({'start':t_ELM_start.data, 'end':t_ELM_end.data}, index=[0])
    else:
        t_ELM = pd.DataFrame({'start':t_ELM_start.data, 'end':t_ELM_end.data})

    try:
        len(t_H_mode_start.data)
    except:
        t_H_mode = pd.DataFrame({'start':t_H_mode_start.data, 'end':t_H_mode_end.data}, index=[0])
    else:
        t_H_mode = pd.DataFrame({'start':t_H_mode_start.data, 'end':t_H_mode_end.data})


    for H_mode in t_H_mode.values:
        LorH.loc[H_mode[0]:H_mode[1]] = 'H-mode'

    for elm in t_ELM.values:
        LorH.loc[elm[0]:elm[1]] = 'ELM'

    #Discarding pictures without plasma
    discharge_start, discharge_end = img.discharge_duration(shot)
    LorH = LorH[discharge_start : discharge_end]

    #Appending columns with paths of the RIS imgs
    LorH['filename'] = np.array(ris1_names)

    LorH.to_csv(f'./data/LHmode-detection-shots/LHmode-detection-shot{shot}.csv')

Working on shot  19083
Demosaicing images


Saving images:   0%|          | 0/1949 [00:00<?, ?it/s]

Demosaicing images


Saving images:   0%|          | 0/1949 [00:00<?, ?it/s]

Working on shot  19094
Demosaicing images


...

Saving images:   0%|          | 0/1951 [00:00<?, ?it/s]

Working on shot  20009
Demosaicing images


Saving images:   0%|          | 0/3398 [00:00<?, ?it/s]

Demosaicing images


Saving images:   0%|          | 0/3398 [00:00<?, ?it/s]